In [133]:
import math
import random

import gym
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

In [134]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

<h2>Use CUDA</h2>

In [135]:
use_cuda = torch.cuda.is_available()
print(use_cuda)
device   = torch.device("cuda" if use_cuda else "cpu")

True


<h2>Create Environments</h2>

In [136]:
from common.multiprocessing_env import SubprocVecEnv

num_envs = 3
#env_name = "Pendulum-v0"
env_name = "FrozenLake-v0"

# def make_env():
#     def _thunk():
#         env = gym.make(env_name)
#         return env

#     return _thunk

# envs = [make_env() for i in range(num_envs)]
# envs = SubprocVecEnv(envs)

env = gym.make(env_name)

[2018-06-14 02:04:09,347] Making new env: FrozenLake-v0


<h2>Neural Network</h2>

In [137]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0., std=0.1)
        nn.init.constant_(m.bias, 0.1)


class ActorCritic(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_size, std=0.0):
        super(ActorCritic, self).__init__()
        
        self.critic = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        
        self.actor = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            #nn.Tanh(),
            nn.Linear(hidden_size, num_outputs),
        )
        self.log_std = nn.Parameter(torch.ones(1, num_outputs) * std)
        
        self.apply(init_weights)
        
    def forward(self, x):
        value = self.critic(x)
        mu    = self.actor(x)
        std   = self.log_std.exp().expand_as(mu)
        dist  = Normal(mu, std)
        return dist, value

In [138]:
def state_fix(evir,states):
    new_state = np.zeros(evir.observation_space.n)
    new_state[states] =  1
    return new_state

In [139]:
def plot(frame_idx, rewards):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
    plt.plot(rewards)
    plt.show()
    
def test_env(vis=False):
    state = env.reset()
    state = state_fix(env,state)
    if vis: env.render()
    done = False
    total_reward = 0
    while not done:
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        dist, _ = model(state)
        action = np.argmax(dist.sample().cpu().numpy()[0])
        next_state, reward, done, _ = env.step(action)
        state = state_fix(env,next_state)
        if vis: env.render()
        total_reward += reward
    return total_reward

<h1>High-Dimensional Continuous Control Using Generalized Advantage Estimation</h1>
<h3><a href="https://arxiv.org/abs/1506.02438">Arxiv</a></h3>

In [140]:
def compute_gae(next_value, rewards, masks, values, gamma=0.99, tau=0.95):
    values = values + [next_value]
    gae = 0
    returns = []
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * tau * masks[step] * gae
        returns.insert(0, gae + values[step])
    return returns

In [141]:
print(env.observation_space.n)
print(env.action_space.n)

num_inputs  = env.observation_space.n
num_outputs = env.action_space.n

#Hyper params:
hidden_size = 10 #256
lr          = 3e-2
num_steps   = 25

model = ActorCritic(num_inputs, num_outputs, hidden_size).to(device)
optimizer = optim.Adam(model.parameters())

print(model)


16
4
ActorCritic(
  (critic): Sequential(
    (0): Linear(in_features=16, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=1, bias=True)
  )
  (actor): Sequential(
    (0): Linear(in_features=16, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=4, bias=True)
  )
)


In [142]:
max_frames   = 100000
frame_idx    = 0
test_rewards = []

In [143]:
state = env.reset()
state = state_fix(env,state)
done = False

while frame_idx < max_frames:

    log_probs = []
    values    = []
    rewards   = []
    masks     = []
    entropy = 0

    #for _ in range(num_steps):
    while not done:
        print("start")
        state = torch.FloatTensor(state).to(device).unsqueeze(0)
        dist, value = model(state)
        
        action = dist.sample()
        print("action", action.cpu().numpy())
        #np.absolute(np.trunc(action.cpu().numpy()[0][0]))
        action_value = np.argmax(action.cpu().numpy()[0])
        print("action_value", action_value)
        next_state, reward, done, _ = env.step(action_value)
       
        
        # Log probablity of action
        log_prob = dist.log_prob(action)
        entropy += dist.entropy().mean()
        
        log_probs.append(log_prob)
        values.append(value)
        #List of rewards
        rewards.append(reward)
        masks.append(1 - done)
        
        state = next_state
        state = state_fix(env,state)
        print(state)
        
        frame_idx += 1
        
        if frame_idx % 1000 == 0:
            test_rewards.append(np.mean([test_env() for _ in range(10)]))
            plot(frame_idx, test_rewards)
    
 
    next_state = state_fix(env,next_state)
    next_state = torch.FloatTensor(next_state).to(device).unsqueeze(0)
    _, next_value = model(next_state)
    
    returns = compute_gae(next_value, rewards, masks, values)
    log_probs = torch.cat(log_probs)
    returns   = torch.cat(returns).detach()
    values    = torch.cat(values)
    print(values)
    #Added b/c it is discrete 
    state = env.reset()
    state = state_fix(env,state)

    advantage = returns - values

    actor_loss  = -(log_probs * advantage.detach()).mean()
    critic_loss = advantage.pow(2).mean()

    loss = actor_loss + 0.5 * critic_loss - 0.001 * entropy

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

values = []
for state in range(0,16):
    state = state_fix(env,state)
    state = torch.FloatTensor(state).to(device).unsqueeze(0)
    dist, value = model(state)
    values.append(value)
print("end_Values", values)
    
    

start
action [[-0.93745714 -0.97966605 -0.6614332   1.3932328 ]]
action_value 3
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
start
action [[-1.4936392   1.4793581   0.7289047  -0.06263874]]
action_value 1
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
start
action [[ 0.4788017  -0.85500044  0.00767577 -0.9956509 ]]
action_value 0
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
start
action [[ 0.44722337  0.6513761  -0.19235829  0.55283254]]
action_value 1
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
tensor([[ 0.0837],
        [ 0.0837],
        [ 0.0837],
        [ 0.1408]], device='cuda:0')


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
test_env(True)